# Generative Agents in LangChain

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Pam"  # The name you want to use when interviewing the agent.

# Local Llama3 
LLM = ChatOllama(
    model="llama3",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=4096 # officailly 1024
    )

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriever, which fetches documents based on salience, recency, and importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

In [5]:
import numpy as np

import faiss


def score_normalizer(val: float) -> float:
    ret = 1.0 - 1.0 / (1.0 + np.exp(val))
    #print("val: "+str(float(val))+"_"+"ret: "+str(ret))
    return ret

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OllamaEmbeddings(model="llama3")

    # Automatically determine the size of the embeddings
    test_embedding = embeddings_model.embed_query("test query")
    embedding_size = len(test_embedding)
    
    # Initialize the vectorstore as empty
    index = faiss.IndexFlatL2(embedding_size)
    
    # Initialize FAISS vector store
    vectorstore = FAISS(
        embeddings_model,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=score_normalizer,
        normalize_L2=True
    )
    
    # Create and return the retriever
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, 
        other_score_keys=["importance"], 
        k=15
    )

In [6]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    memory=tommies_memory,
)

In [7]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Based on the provided statements, I would summarize Tommie's core characteristics as:

* Honest
* Loyal
* Kind


In [8]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]

for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [9]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Based on the statements, Tommie's core characteristics are:

* He has a sentimental attachment to his childhood memories (remembers his dog Bruno)
* He is physically tired from driving (feels tired from driving so far)
* He is hungry (is hungry)
* He is trying to rest or sleep (tries to get some rest)

These characteristics suggest that Tommie is someone who values nostalgia, has a physical presence that can be affected by his actions (in this case, driving), and has basic needs like hunger.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [10]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [11]:
interview_agent(tommie, "What are you looking forward to doing today?")

/Users/motif/anaconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/motif/anaconda3/envs/langchain/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'Tommie said "Honestly, Pam, I\'m just looking forward to getting some rest and maybe finding a job. Driving all day has left me exhausted. And, you know, it\'s always nice to have something to look forward to."'

In [12]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m just worried about finding a job and getting settled in this new place. The noise from the road at night is already keeping me up, and I\'m not sure how I\'ll adjust to it. Plus, being hungry isn\'t helping my mood either."'

## Step through the day's observations.

In [13]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [14]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. DO NOTHING: Tommie is too tired and hungry to react to the noise from the construction site, and his primary concerns are finding a job and getting settled in the new home.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. DO NOTHING: Tommie is too tired and hungry to react to the noise from the construction site, and his primary concerns are finding a job and getting settled in the new home.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. DO NOTHING: Tommie is too tired and hungry to worry about forgetting coffee filters, and his primary concerns are finding a job and getting settled in the new home.
Tommie finally finds the filters and makes himself a cup of coffee. DO NOTHING: Tommie is too tired and hungry to worry about finally finding the coffee filters, and his primary concerns are still finding a job and getting set

## Interview after the day

In [15]:
interview_agent(tommie, "Tell me about how your day has been going")

"Based on Tommie's characteristics and recent observations, I think he would respond to Pam's question with a mix of his emotions and experiences. Here's what he might say:"

In [16]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "Oh, I love coffee! It\'s one of my favorite things. You know, it reminds me of my childhood and spending time with Bruno, my old dog. He used to love going on walks with me while I was sipping on a cup of coffee. Those were some great times."'

In [17]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, Bruno was my childhood dog. He was a golden retriever and just the sweetest thing. I have so many fond memories of playing with him in the backyard, going on walks together, and just cuddling up with him on the couch. He was more than just a pet to me; he was like a member of the family. Even thinking about him now makes me feel all warm and fuzzy inside."'

## Adding Multiple Characters

Let's add a second character to have a conversation with Tommie. Feel free to configure different traits.

In [18]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [19]:
from datetime import datetime, timedelta

yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [20]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Based on the given statements, Eve's core characteristics can be summarized as:

* Wakes up early and starts her day with an alarm
* Enjoys playing tennis with friends before work
* Has a social life and interacts with colleagues at work
* Is observant and overhears conversations about coworkers (in this case, Tommie)
* Values helping others and is willing to lend a hand to a coworker in need


## Pre-conversation interviews


Let's "Interview" Eve before she speaks with Tommie.

In [21]:
interview_agent(eve, "How are you feeling about today?")

"Eve said I'm doing well, thanks for asking! Just got back from a game of tennis with my friend Xu and had a great time. Then I helped out a coworker on a task, so that's been keeping me busy."

In [22]:
interview_agent(eve, "What do you know about Tommie?")

"Eve said I've actually overheard some colleagues talking about Tommie being hard to work with. Nothing specific, just a general impression that they can be challenging to collaborate with."

In [23]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

"Eve said I think it's interesting that he's looking for a job. From what I've overheard, some colleagues have mentioned that he can be challenging to work with. Maybe we could ask him about his strengths and weaknesses, or what kind of work environment he prefers? That might give us a better idea of how to support him in his job search."

In [24]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "That\'s a good point about him being anxious. I\'ll definitely try to keep the conversation going and ask some questions. Do you think he\'d be open to getting feedback on his strengths and weaknesses from us?"'

## Dialogue between Generative Agents

Generative agents are much more complex when they interact with a virtual environment or with each other. Below, we run a simple conversation between Tommie and Eve.

In [25]:
from typing import List

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [26]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

Eve Eve's eyes light up with curiosity as she leans forward slightly, interested in hearing Tommie's questions and sharing her own experience.
Based on Tommie's characteristics and recent observations, I think he would:
Based on the given statements, here's what Eve might say:


## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

In [27]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Based on the given statements, Tommie's core characteristics can be summarized as:

1. **Emotional**: Tommie is a sentimental person who often feels comfort and nostalgia when thinking about his childhood memories with Bruno, his old dog.
2. **Hopeful**: Tommie has a positive outlook and is eager to learn more about job opportunities, as seen in his reactions to conversations about job openings.
3. **Anxious**: Tommie tends to feel anxious or worried about the lack of job offers, financial stability, and uncertainty about his future.
4. **Comfort-seeking**: Tommie often seeks comfort and solace in nostalgic memories or interactions with others that remind him of happy times (e.g., playing frisbee, talking to friends).
5. **Resilient**: Despite experiencing setbacks (e.g., not getting job offers, getting hurt while playing), Tommie seems to bounce back and maintain a positive attitude.

These characteristics provide 

In [28]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Based on the given statements, Eve's core characteristics can be summarized as:

* Curious and interested in hearing others' thoughts and experiences (observed Tommie's message and leaned forward slightly)
* Helpful and willing to assist others (helped a coworker with a task)
* Friendly and sociable (played tennis with friend Xu and had a great time)
* Considerate of others' feelings and needs (asked about Tommie's anxiety and willingness to receive feedback)
* Observant and aware of her surroundings (overheard colleagues talking about Tommie being hard to work with)

These characteristics suggest that Eve is a friendly, helpful, and curious person who values building relationships and supporting others.


In [29]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "Oh, it was really nice. I love talking about Bruno and my childhood memories. It\'s always so comforting to think about those times. But, you know, it\'s also a bit bittersweet because it makes me realize how much I miss having a pet like that in my life now."'

In [30]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "That\'s really interesting. I think it\'s great that he\'s taking steps to find a new job. What are some of his top priorities in terms of what he\'s looking for in a work environment?"'

In [31]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "I think I wish I had asked him more questions about his goals and aspirations. I was so focused on sharing my own experience that I didn\'t really give him a chance to open up and share his thoughts and feelings. Maybe if I had done that, we could have had a more meaningful conversation and I could have learned more about what he\'s looking for in a new job."'